In [171]:
import pandas as pd
import numpy as np
import nltk

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [46]:
# load google news word2vec
import gensim.downloader as api
# w2v = api.load('word2vec-google-news-300')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pouri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
dataset = pd.read_csv('Data/sentiment140.csv')
dataset

,text,date,user,sentiment,query
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,0,NO_QUERY
1,is upset that he can't update his Facebook by ...,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,0,NO_QUERY
2,@Kenichan I dived many times for the ball. Man...,Mon Apr 06 22:19:53 PDT 2009,mattycus,0,NO_QUERY
3,my whole body feels itchy and like its on fire,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,0,NO_QUERY
4,"@nationwideclass no, it's not behaving at all....",Mon Apr 06 22:19:57 PDT 2009,Karoli,0,NO_QUERY
...,...,...,...,...,...
1599995,Just woke up. Having no school is the best fee...,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,4,NO_QUERY
1599996,TheWDB.com - Very cool to hear old Walt interv...,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,4,NO_QUERY
1599997,Are you ready for your MoJo Makeover? Ask me f...,Tue Jun 16 08:40:49 PDT 2009,bpbabe,4,NO_QUERY
1599998,Happy 38th Birthday to my boo of alll time!!! ...,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,4,NO_QUERY


In [48]:
#remove web addresses, signs
#change to lowercase

import re
import string

def clean_en_text(text):

    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '<URL>', text)
    text = re.sub('@\S+', '<MENTION>', text)
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)

    # removing sign
    text = ''.join([i for i in text if ord(i) not in [33, 34, 35, 36, 37, 38,
                                                      39, 40, 41, 42, 43, 44,
                                                      45, 46, 47, 58, 59, 60,
                                                      61, 62, 63, 64, 91, 92,
                                                      93, 94, 95, 96, 123, 124,
                                                      125, 126, 1548, 1567]])
    return text

dataset['text'] = dataset['text'].apply(clean_en_text)

In [49]:
# Tokenizing

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
dataset['text'] = dataset['text'].apply(lambda x: tokenizer.tokenize(x))

In [6]:
#Lemmatizing
from nltk.stem import WordNetLemmatizer

def lemmatizer(text_list):
    lemm = WordNetLemmatizer()
    words = [lemm.lemmatize(word) for word in text_list]
    return words


dataset['text'] = dataset['text'].apply(lambda x: lemmatizer(x))

In [50]:
#Showing 20 first twits
dataset['text'][0:20]

0     [MENTION, URL, awww, that, s, a, bummer, you, ...
1     [is, upset, that, he, can, t, update, his, fac...
2     [MENTION, i, dived, many, times, for, the, bal...
3     [my, whole, body, feels, itchy, and, like, its...
4     [MENTION, no, it, s, not, behaving, at, all, i...
5                      [MENTION, not, the, whole, crew]
6                                        [need, a, hug]
7     [MENTION, hey, long, time, no, see, yes, rains...
8              [MENTION, nope, they, didn, t, have, it]
9                             [MENTION, que, me, muera]
10     [spring, break, in, plain, city, it, s, snowing]
11                     [i, just, re, pierced, my, ears]
12    [MENTION, i, couldn, t, bear, to, watch, it, a...
13    [MENTION, it, it, counts, idk, why, i, did, ei...
14    [MENTION, i, would, ve, been, the, first, but,...
15    [MENTION, i, wish, i, got, to, watch, it, with...
16    [hollis, death, scene, will, hurt, me, severel...
17                             [about, to, file,

In [8]:
def create_vocabulary(all_text):
    vocabulary = {'PAD': 0, 'END': 1, 'UNK': 2}

    for row in all_text:
        for word in row:
            if word not in vocabulary.keys():
                vocabulary[word] = len(vocabulary.keys())
    return vocabulary


def get_max_length(all_text: pd.DataFrame):
    max_length = 0
    for row in all_text:
        if len(row) > max_length:
            max_length = len(row)
    return max_length

def vectorization(all_text: pd.DataFrame, vocabulary: dict):
    new_embedding = all_text.copy(deep=True)
    for i in range(len(all_text)):
        row = all_text.iloc[i]
        new_embedding.iloc[i] = [vocabulary[token] for token in row]
    return new_embedding

def apply_padding(vectorized_dataset: pd.DataFrame, max_length: int, vocabulary: dict, pad_token: str):
    copy_data = vectorized_dataset.copy(deep=True)
    for i in range(len(vectorized_dataset)):
        row = copy_data.iloc[i]
        padding_token_id = vocabulary[pad_token]
        copy_data.iloc[i] = row + [padding_token_id for _ in range(max_length - len(row))]
    return copy_data

vocabulary = create_vocabulary(dataset['text'])
max_length = get_max_length(dataset['text'])
vectorized_data = vectorization(dataset['text'], vocabulary)
vectorized_padding = apply_padding(vectorized_data, max_length=max_length, vocabulary=vocabulary, pad_token='PAD')
vectorized_padding[:10]

0    [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...
1    [22, 23, 6, 24, 25, 26, 27, 28, 29, 30, 31, 20...
2    [3, 40, 41, 42, 43, 44, 45, 46, 47, 18, 48, 45...
3    [53, 54, 55, 56, 57, 32, 58, 20, 59, 60, 0, 0,...
4    [3, 61, 20, 7, 62, 63, 64, 65, 40, 66, 67, 68,...
5    [3, 62, 45, 54, 75, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
6    [76, 8, 77, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
7    [3, 78, 79, 43, 61, 72, 80, 81, 8, 82, 83, 8, ...
8    [3, 88, 89, 90, 26, 91, 20, 0, 0, 0, 0, 0, 0, ...
9    [3, 92, 93, 94, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: text, dtype: object

In [161]:
from collections import Counter

def encode(tokens, word2idx):
    return [word2idx[token] for token in tokens if token in word2idx]


# Build vocabulary
dataset['tokens'] = dataset['text']
vocab = Counter()
for tokens in dataset['tokens']:
    vocab.update(tokens)

# Create a mapping from word to index
word2idx = {word: idx + 1 for idx, (word, _) in enumerate(vocab.most_common())}  # Start indices from 1
word2idx['<PAD>'] = 0  # Padding token

dataset['encoded'] = dataset['tokens'].apply(lambda x: encode(x, word2idx))
dataset

,text,date,user,sentiment,query,tokens,encoded
0,"[MENTION, URL, awww, that, s, a, bummer, you, ...",Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,0,NO_QUERY,"[MENTION, URL, awww, that, s, a, bummer, you, ...","[2, 40, 490, 17, 13, 5, 1174, 9, 3432, 52, 835..."
1,"[is, upset, that, he, can, t, update, his, fac...",Mon Apr 06 22:19:49 PDT 2009,scotthamilton,0,NO_QUERY,"[is, upset, that, he, can, t, update, his, fac...","[10, 776, 17, 88, 31, 15, 543, 187, 532, 127, ..."
2,"[MENTION, i, dived, many, times, for, the, bal...",Mon Apr 06 22:19:53 PDT 2009,mattycus,0,NO_QUERY,"[MENTION, i, dived, many, times, for, the, bal...","[2, 1, 51971, 314, 355, 12, 4, 1271, 1666, 3, ..."
3,"[my, whole, body, feels, itchy, and, like, its...",Mon Apr 06 22:19:57 PDT 2009,ElleCTF,0,NO_QUERY,"[my, whole, body, feels, itchy, and, like, its...","[6, 428, 801, 481, 2864, 8, 38, 72, 16, 1143]"
4,"[MENTION, no, it, s, not, behaving, at, all, i...",Mon Apr 06 22:19:57 PDT 2009,Karoli,0,NO_QUERY,"[MENTION, no, it, s, not, behaving, at, all, i...","[2, 41, 7, 13, 27, 10059, 26, 35, 1, 21, 594, ..."
...,...,...,...,...,...,...,...
1599995,"[just, woke, up, having, no, school, is, the, ...",Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,4,NO_QUERY,"[just, woke, up, having, no, school, is, the, ...","[23, 345, 33, 184, 41, 146, 10, 4, 176, 191, 223]"
1599996,"[thewdb, com, very, cool, to, hear, old, walt,...",Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,4,NO_QUERY,"[thewdb, com, very, cool, to, hear, old, walt,...","[267173, 587, 122, 201, 3, 282, 230, 12660, 39..."
1599997,"[are, you, ready, for, your, mojo, makeover, a...",Tue Jun 16 08:40:49 PDT 2009,bpbabe,4,NO_QUERY,"[are, you, ready, for, your, mojo, makeover, a...","[39, 9, 209, 12, 47, 7606, 8092, 615, 18, 12, ..."
1599998,"[happy, birthday, to, my, boo, of, alll, time,...",Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,4,NO_QUERY,"[happy, birthday, to, my, boo, of, alll, time,...","[121, 269, 3, 6, 478, 14, 4973, 55, 11852, 715..."


In [168]:
#label Encoding
max_len = max(dataset['encoded'].apply(len))

def pad_sequence(seq, max_len):
    return seq + [word2idx['<PAD>']] * (max_len - len(seq))

dataset['padded'] = dataset['encoded'].apply(lambda x: pad_sequence(x, max_len))
dataset['padded'][:10]

0    [2, 40, 490, 17, 13, 5, 1174, 9, 3432, 52, 835...
1    [10, 776, 17, 88, 31, 15, 543, 187, 532, 127, ...
2    [2, 1, 51971, 314, 355, 12, 4, 1271, 1666, 3, ...
3    [6, 428, 801, 481, 2864, 8, 38, 72, 16, 1143, ...
4    [2, 41, 7, 13, 27, 10059, 26, 35, 1, 21, 594, ...
5    [2, 27, 4, 428, 2156, 0, 0, 0, 0, 0, 0, 0, 0, ...
6    [94, 5, 913, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7    [2, 153, 168, 55, 41, 69, 155, 2617, 5, 251, 1...
8    [2, 786, 77, 145, 15, 20, 7, 0, 0, 0, 0, 0, 0,...
9    [2, 2454, 18, 106981, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: padded, dtype: object

In [ ]:
#label Encoding
label2idx = {label: idx for idx, label in enumerate(dataset['sentiment'].unique())}
dataset['label_idx'] = dataset['sentiment'].map(label2idx)

In [172]:
# Split the data
from sklearn.model_selection import train_test_split


train_df, temp_df = train_test_split(dataset, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = TextDataset(train_df['padded'].tolist(), train_df['label_idx'].tolist())
val_dataset = TextDataset(val_df['padded'].tolist(), val_df['label_idx'].tolist())
test_dataset = TextDataset(test_df['padded'].tolist(), test_df['label_idx'].tolist())

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [173]:
iter(train_loader).__next__()

[tensor([[   2,  163,    9,  ...,    0,    0,    0],
         [   2, 3190,  662,  ...,    0,    0,    0],
         [  19,   72,   64,  ...,    0,    0,    0],
         ...,
         [   2,   32,    7,  ...,    0,    0,    0],
         [   1,   23, 3268,  ...,    0,    0,    0],
         [   2,    9,  319,  ...,    0,    0,    0]]),
 tensor([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
         1, 1, 0, 0, 0, 1, 1, 1])]

In [175]:
import torch.nn as nn

class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1):
        super(TextRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=word2idx['<PAD>'])
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

vocab_size = len(word2idx)
embed_size = 100
hidden_size = 128
output_size = len(label2idx)
num_layers = 2

model = TextRNN(vocab_size, embed_size, hidden_size, output_size, num_layers)

In [176]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for texts, labels in train_loader:
        optimizer.zero_grad()
        output = model(texts)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in val_loader:
            output = model(texts)
            loss = criterion(output, labels)
            val_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {accuracy}%')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
